# Program Executions in Suspicious Folders
Detects program executions in suspicious non-program folders related to malware or hacking activity

## Rule Content
```
- title: Program Executions in Suspicious Folders
  id: a39d7fa7-3fbd-4dc2-97e1-d87f546b1bbc
  status: experimental
  description: Detects program executions in suspicious non-program folders related
    to malware or hacking activity
  references:
  - Internal Research
  date: 2018/01/23
  author: Florian Roth
  logsource:
    product: linux
    service: auditd
    category: null
  detection:
    selection:
      type: SYSCALL
      exe:
      - /tmp/*
      - /var/www/*
      - /home/*/public_html/*
      - /usr/local/apache2/*
      - /usr/local/httpd/*
      - /var/apache/*
      - /srv/www/*
      - /home/httpd/html/*
      - /srv/http/*
      - /usr/share/nginx/html/*
      - /var/lib/pgsql/data/*
      - /usr/local/mysql/data/*
      - /var/lib/mysql/*
      - /var/vsftpd/*
      - /etc/bind/*
      - /var/named/*
    condition: selection
  falsepositives:
  - Admin activity (especially in /tmp folders)
  - Crazy web applications
  level: medium

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='(type:"SYSCALL" AND exe.keyword:(\/tmp\/* OR \/var\/www\/* OR \/home\/*\/public_html\/* OR \/usr\/local\/apache2\/* OR \/usr\/local\/httpd\/* OR \/var\/apache\/* OR \/srv\/www\/* OR \/home\/httpd\/html\/* OR \/srv\/http\/* OR \/usr\/share\/nginx\/html\/* OR \/var\/lib\/pgsql\/data\/* OR \/usr\/local\/mysql\/data\/* OR \/var\/lib\/mysql\/* OR \/var\/vsftpd\/* OR \/etc\/bind\/* OR \/var\/named\/*))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()